In [1]:
import cv2


cap = cv2.VideoCapture("videos/me_boxing.mp4")
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    fgmask = fgbg.apply(frame)

    # Further processing and limb tracking can be added her
    
    cv2.imqhow("Original Frame", frame)
    cv2.imshow("Foreground Mask", fgmask)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
import cv2

cap = cv2.VideoCapture("videos/me_boxing.mp4")

prev_centroid = None

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame = cv2.resize(frame, (640, 480))
    frame_blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    frame_hsv = cv2.cvtColor(frame_blurred, cv2.COLOR_BGR2HSV)

    lower_color = (0, 20, 100)
    upper_color = (30, 255, 255)

    mask = cv2.inRange(frame_hsv, lower_color, upper_color)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest_contour)
        if M["m00"] > 0:
            centroid_x = int(M["m10"] / M["m00"])
            centroid_y = int(M["m01"] / M["m00"])

            if prev_centroid:
                displacement = (
                    (centroid_x - prev_centroid[0]) ** 2
                    + (centroid_y - prev_centroid[1]) ** 2
                ) ** 0.5
                print(f"Displacement: {displacement}")

            prev_centroid = (centroid_x, centroid_y)

    cv2.drawContours(frame, [largest_contour], -1, (0, 255, 0), 2)
    cv2.circle(frame, (centroid_x, centroid_y), 5, (0, 0, 255), -1)

    cv2.imshow("Processed Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()